In [1]:
from tensorflow.keras import backend as K
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np


# GPU config if needed
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Recurrent neural network based on predictive coding properties

In this simulation, we will use task of [Abdullahi et. al 2020] by increasing temporal resolution from one image per step to 10 iterations for each input.

In [ ]:
# TB
# dir(tf.keras.layers.LSTM)

In [2]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()

def create_serial_dataset(x=None, y=None, n=100, length=10, frames=10):

    X = np.zeros([n, length*frames, x.shape[1], x.shape[2]])
    Y = np.zeros([n, length*frames, 10])

    for i in range(n):

        k = np.random.randint(0, 1000, size=(length))
        for j in range(length):
          
            X[i, j*frames:j*frames+frames, :, :] = x[k[j], :, :]
            Y[i, j*frames:j*frames+frames, y[k[j]]] = 1
          
    return X.reshape(n, length*frames, x.shape[1] * x.shape[2]), Y

Xn, Yn = create_serial_dataset(x_train, y_train, n=100, length=10, frames=20)
X = Xn[:10, :, :]
Y = Yn[:10, :, :]
X.shape, Y.shape

11493376/11490434 [==============================] - 0s 0us/step


((10, 200, 784), (10, 200, 10))

In [3]:
class RNNModel1(tf.keras.Model):
    def __init__(self):
        super(RNNModel1, self).__init__()
        self = tf.keras.Sequential()
        self.add(tf.keras.layers.InputLayer(input_shape=(None, 784)))
        self.add(tf.keras.layers.LSTM(
          units=512,
          recurrent_dropout=0.2,
          return_sequences=True,
          # return_state=True
        ))
        self.add(tf.keras.layers.Dense(units=10))
        return
        
    def call(self, x):
        return self(x)

    def get_state(self):
        return

class RNNModel2(tf.keras.Model):
    def __init__(self, units, vector_dim):
        super(RNNModel2, self).__init__()
        self.units = units
        # self.embedding = tf.keras.layers.Embedding(vector_dim, 10)
        self.rnn = tf.keras.layers.LSTM(self.units,
                                      return_sequences=True,
                                      return_state=True,
                                      recurrent_initializer='glorot_uniform')
        
        self.fc1 = tf.keras.layers.Dense(vector_dim)
        return

    def call(self, x):
        # x = self.embedding(x)
        x2, state, x3 = self.rnn(x)

        x = self.fc1(x2)
        return x, state

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))


In [4]:
def EnergyCostLoss(y_true, y_pred):
    error = y_pred - y_true
    return K.mean(K.square(error) + 2*K.mean(K.abs(y_pred)))

In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(None, 784)))
model.add(tf.keras.layers.LSTM(
  units=512,
  recurrent_dropout=0.2,
  return_sequences=True,
  # return_state=True
))
model.add(tf.keras.layers.Dense(units=10))
# model = RNNModel1()
model.compile(
  loss=EnergyCostLoss,
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.003)
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 512)         2656256   
_________________________________________________________________
dense (Dense)                (None, None, 10)          5130      
Total params: 2,661,386
Trainable params: 2,661,386
Non-trainable params: 0
_________________________________________________________________


In [6]:
history = model.fit(
    x=X, y=Y,
    epochs=50,
    batch_size=5,
    validation_split=0.0,
    verbose=1,
    shuffle=False
)

Epoch 1/50
2/2 [==============================] - 5s 235ms/step - loss: 2.1454
Epoch 2/50
2/2 [==============================] - 0s 257ms/step - loss: 1.4483
Epoch 3/50
2/2 [==============================] - 1s 251ms/step - loss: 0.9194
Epoch 4/50
2/2 [==============================] - 0s 242ms/step - loss: 0.6969
Epoch 5/50
2/2 [==============================] - 0s 235ms/step - loss: 0.6315
Epoch 6/50
2/2 [==============================] - 0s 241ms/step - loss: 0.5338
Epoch 7/50
2/2 [==============================] - 0s 242ms/step - loss: 0.4835
Epoch 8/50
2/2 [==============================] - 0s 237ms/step - loss: 0.4554
Epoch 9/50
2/2 [==============================] - 0s 237ms/step - loss: 0.4194
Epoch 10/50
2/2 [==============================] - 0s 232ms/step - loss: 0.3976
Epoch 11/50
2/2 [==============================] - 0s 243ms/step - loss: 0.3813
Epoch 12/50
2/2 [==============================] - 0s 237ms/step - loss: 0.3554
Epoch 13/50
2/2 [==============================] 

In [7]:
# Median layer model
L1 = K.function([model.layers[0].input],
                                  [model.layers[0].output])


In [11]:
means1 = []

for k in range(10):
    l, r = k, k+1
    l_pred = L1(Xn[l:r, :, :])[0]
    y_pred = model.predict(Xn[l:r, :, :])

    # for i in range(7):
        # plt.plot(l_pred[0][0, :, i])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means1.append(np.mean(np.mean(l_pred)))

In [12]:
means2 = []

for k in range(20, 50):
    l, r = k, k+1
    l_pred = L1(Xn[l:r, :, :])[0]
    y_pred = model.predict(Xn[l:r, :, :])

    # for i in range(7):
    #     plt.plot(l_pred[0, i, :])
    # for i in range(10):
    # plt.plot(y_pred[0, :, 6])
    # for i in range(0, 100, 10):
    #     print(Yn[l:r, i, :])
    means2.append(np.mean(np.mean(l_pred)))

In [13]:
print(np.mean(means1), np.mean(means2))

0.01985993 0.02046866
